In [2]:
from keras import Input
from keras.layers import Dense, Embedding, LSTM, concatenate, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential, Model
import numpy as np

In [6]:
# simple NN
model = Sequential([
    Dense(32, activation='relu', input_shape=(64,)),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
])

In [3]:
# equivalent NN using functional API
input_tensor = Input(shape=(64,))
x = Dense(32, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)
output_tensor = Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## Multiple input models
Network which gets question and reference text as an input and outputs an answer to that question.

In [ ]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = LSTM(16)(embedded_question)

concatenated = concatenate([encoded_text, encoded_question], axis=-1)

answer = Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer) # you can also pass a dictionary

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [7]:
num_samples = 1000
max_len = 100

In [20]:
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_len))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_len))
answers = np.random.randint(0, 2, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128, verbose=0)
# model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

NameError: name 'text_vocabulary_size' is not defined

## Multiple output models
Network that takes as an input some statement from social media and predict age, income and gender of an author.

In [29]:
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = Embedding(vocabulary_size, 256)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)

age_pred = Dense(1, name='age')(x)
income_pred = Dense(num_income_groups, activation='softmax', name='income')(x)
gender_pred = Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_pred, income_pred, gender_pred])
model.compile(optimizer='adam', metrics=['accuracy'], loss=['mae', 'sparse_categorical_crossentropy', 'binary_crossentropy']) # you can also use dictionary with keys: age, gender, income

In [31]:
posts = np.random.randint(1, vocabulary_size, size=(num_samples,500))
target_age = np.random.randint(18, 99, size=num_samples)
target_income = np.random.randint(1, 10, size=num_samples)
target_gender = np.random.randint(0, 2, size=num_samples)

model.fit(posts, [target_age, target_income, target_gender], batch_size=128, epochs=3) # or with dictionary

Epoch 1/3
8/8 [==============================] - 4s 476ms/step - loss: 23.9978 - age_loss: 20.4890 - income_loss: 2.7027 - gender_loss: 0.8061 - age_accuracy: 0.0000e+00 - income_accuracy: 0.1260 - gender_accuracy: 0.5030
Epoch 2/3
8/8 [==============================] - 4s 474ms/step - loss: 20.2176 - age_loss: 16.8574 - income_loss: 2.5867 - gender_loss: 0.7735 - age_accuracy: 0.0000e+00 - income_accuracy: 0.1040 - gender_accuracy: 0.5110
Epoch 3/3
8/8 [==============================] - 4s 482ms/step - loss: 13.7427 - age_loss: 10.6099 - income_loss: 2.3650 - gender_loss: 0.7678 - age_accuracy: 0.0000e+00 - income_accuracy: 0.1120 - gender_accuracy: 0.5070


## Directed acyclic graphs